# Import Libraries

In [1]:
import numpy as np
import wfdb
import hrvanalysis 
import matplotlib.pyplot as plt
from wfdb import get_record_list,plot_wfdb
from wfdb.io import rdheader,rdann,rdrecord
from wfdb.processing import normalize_bound
import ecgdetectors
import scipy
from scipy import signal
#import emd

ImportError: cannot import name 'LombScargle' from 'astropy.stats' (c:\Users\Prime\AppData\Local\Programs\Python\Python312\Lib\site-packages\astropy\stats\__init__.py)

# Reading ECG record (data, header, annotation)

# Database detail

In [ ]:
get_record_list('apnea-ecg/1.0.0/',records='all')

In [ ]:
ecg_record_head=rdheader(record_name=r'D:\sleep_apnea\apnea-ecg-database-1.0.0/a01',pn_dir=None,rd_segments=True)
ecg_record_ann=rdann(record_name=r'D:\sleep_apnea\apnea-ecg-database-1.0.0/a01',extension='apn',sampto=120000,pn_dir=None)
ecg_record_sig=rdrecord(record_name=r'D:\sleep_apnea\apnea-ecg-database-1.0.0/a01',sampto=120000,pn_dir=None)

In [ ]:
display(ecg_record_head.__dict__)

In [ ]:
display(ecg_record_ann.__dict__)

In [ ]:
display(ecg_record_sig.__dict__)

In [ ]:
ecg_record_sig.p_signal

In [ ]:
ecg_record_sig=rdrecord(record_name=r'D:\sleep_apnea\apnea-ecg-database-1.0.0/a01',pn_dir=None,sampfrom=0,sampto=7000)
ecg_record_ann=rdann(record_name=r'D:\sleep_apnea\apnea-ecg-database-1.0.0/a01',extension='apn',sampto=7000,pn_dir=None)
ecg_record_head=rdheader(record_name=r'D:\sleep_apnea\apnea-ecg-database-1.0.0/a01',pn_dir=None,rd_segments=True)

# Visualization of ECG record

In [ ]:
plot_wfdb(ecg_record_sig,title='Record a01 from Physionet Apnea ECG',figsize=[15,10], ecg_grids='all')

# Display record with annotation 

In [ ]:
plot_wfdb(record=ecg_record_sig, annotation=ecg_record_ann, plot_sym=True, title='Record a01 from Physionet Apnea ECG',figsize=(15,10))

# Reading ECG signal

In [ ]:
signals, fields = wfdb.rdsamp(r'D:\sleep_apnea\apnea-ecg-database-1.0.0/a01',pn_dir=None,sampfrom= 0,sampto=6000)
print(signals)
print(fields)

In [ ]:
fs=fields['fs']
T=1/fs ;
t=np.arange(0,fields['sig_len'])*T
plt.figure(figsize=(15,10))
plt.plot(t,signals)
plt.title('ECG signal')
plt.xlabel('time(ms)')
plt.ylabel('Amplitude(mv)')
plt.show()


In [ ]:

ecg_norm=(signals-np.min(signals))/np.max(signals)
plt.figure(figsize=(15,10))
plt.plot(t,ecg_norm)
plt.title('Normalized ECG signal')
plt.xlabel('time(ms)')
plt.ylabel('Amplitude(mv)')
plt.show()


In [ ]:
import pandas as pa
sigl=pa.DataFrame(signals)[0].values

In [ ]:
from ecgdetectors import Detectors

In [ ]:
detectors = Detectors(fs)

In [ ]:
r_peaks = detectors.pan_tompkins_detector(sigl)#(np.transpose(ecg_norm))
#r_p=[n-5 for n in r_peaks]
r_p=r_peaks

In [ ]:
r_p

In [ ]:
plt.figure(figsize=(15,10))
plt.plot(t,ecg_norm)
plt.plot(t[r_p],ecg_norm[r_p],'ro')
plt.title('Rpeak detection using pan-tomkins method')
plt.show()

In [ ]:
from PyEMD import EMD,Visualisation,CEEMDAN
emd = EMD()
IMFs = emd(sigl,t)
imfs, res = emd.get_imfs_and_residue()

In [ ]:
vis = Visualisation()
vis.plot_imfs(imfs=imfs, residue=res, t=t, include_residue=True)
#vis.plot_instant_freq(t, imfs=imfs)
vis.show()

In [ ]:
from scipy.signal import hilbert,find_peaks

In [ ]:
def non_linear(s_1):
    analytic_signal = hilbert(s_1)
    amplitude_envelope = np.abs(analytic_signal)
    #plt.figure(figsize=(10,15))
    #plt.subplot(121),plt.plot(s_1)
    #plt.subplot(122),plt.plot(amplitude_envelope)
    #plt.show()
    return amplitude_envelope

In [ ]:
f_1=non_linear(IMFs[1,:])
plt.figure(figsize=(15,10))
plt.plot(t,f_1)
plt.show()
s_1=non_linear(IMFs[2,:])
plt.figure(figsize=(15,10))
plt.plot(t,s_1)
plt.show()
t_1=non_linear(IMFs[3,:])
plt.figure(figsize=(15,10))
plt.plot(t,t_1)
plt.show()
fr_1=non_linear(IMFs[4,:])
s_non=f_1+s_1+t_1;


In [ ]:
M=10
u=f_1+s_1+t_1
va=np.zeros(len(f_1))
for i in range(M,len(IMFs[1,:])-M):
    tem=np.mean(u[i-M:i])
    va[i]=tem


In [ ]:
plt.figure(figsize=(15,10))
plt.plot(va)
plt.show()

In [ ]:
plt.figure(figsize=(15,10))
plt.plot(t,s_non)
plt.show()

In [ ]:
peaks, _ = find_peaks(sigl, height=np.max(sigl)/2)
plt.figure(figsize=(15,10))
plt.plot(t,ecg_norm)
plt.plot(t[peaks],ecg_norm[peaks],'ro')
plt.title('Rpeak detection using EMD method')
plt.show()

In [ ]:
peaks

In [ ]:
rr=np.diff(t[peaks])

60/rr[1]

In [ ]:
plt.figure(figsize=(20,7))
plt.plot(np.cumsum(rr), rr, label="RR-interval", color="#A651D8")

plt.title('RR interval')
plt.xlabel('Time(ms)')
plt.ylabel('RR interval(s)')
plt.show()

In [ ]:
from hrvanalysis import remove_outliers, remove_ectopic_beats, interpolate_nan_values

In [ ]:
rr_intervals_list =rr

rr_intervals_without_outliers = remove_outliers(rr_intervals=rr_intervals_list,low_rri=np.mean(rr) - 2 * np.std(rr), high_rri=np.mean(rr) + 2 * np.std(rr))

interpolated_rr_intervals = interpolate_nan_values(rr_intervals=rr_intervals_without_outliers,interpolation_method="cubic")

nn_intervals_list = remove_ectopic_beats(rr_intervals=interpolated_rr_intervals, method="malik")

interpolated_nn_intervals = interpolate_nan_values(rr_intervals=nn_intervals_list)

In [ ]:
len(nn_intervals_list)

In [ ]:
plt.figure(figsize=(20,7))
plt.subplot(122)
plt.plot(np.cumsum(rr), interpolated_nn_intervals, label="RR-interval", color='red')

plt.title('NN interval')
plt.xlabel('Time(ms)')
plt.ylabel('NN interval(s)')
plt.ylim(0,2)
plt.subplot(121)

plt.plot(np.cumsum(rr), rr, label="RR-interval", color="green")
plt.ylim(0,2)
plt.title('RR interval')
plt.xlabel('Time(ms)')
plt.ylabel('RR interval(s)')
plt.show()

In [ ]:
from hrvanalysis import get_frequency_domain_features,get_csi_cvi_features,get_geometrical_features,get_poincare_plot_features

In [ ]:
# time domain features

length_int = len(nn_intervals_list) 
diff_nni = np.diff(nn_intervals_list)
mean_nni=np.mean(nn_intervals_list)
median_nni = np.median(nn_intervals_list)
range_nni = max(nn_intervals_list) - min(nn_intervals_list)
sdsd = np.std(diff_nni)
rmssd = np.sqrt(np.mean(diff_nni ** 2))
nni_50 = sum(np.abs(diff_nni) > 50)
pnni_50 = 100 * nni_50 / length_int
nni_20 = sum(np.abs(diff_nni) > 20)
pnni_20 = 100 * nni_20 / length_int
cvsd = rmssd / mean_nni
sdnn = np.std(nn_intervals_list, ddof=1)  
cvnni = sdnn / mean_nni

heart_rate_list = np.divide(60, nn_intervals_list)
mean_hr = np.mean(heart_rate_list)
min_hr = min(heart_rate_list)
max_hr = max(heart_rate_list)
std_hr = np.std(heart_rate_list)

In [ ]:
from scipy.interpolate import interp1d
from scipy.integrate import trapz

In [ ]:
nn_intervals_list

In [ ]:
x = np.cumsum(interpolated_nn_intervals) 
f = interp1d(x,interpolated_nn_intervals, kind='cubic')
fs = 3.0
steps = 1 / fs
xx = np.arange(1, np.max(x), steps)
rr_interpolated = f(xx)

In [ ]:
plt.figure(figsize=(20,15))
plt.subplot(211)
plt.plot(x,nn_intervals_list,color='green')
plt.title('RR-interval')

plt.subplot(212)
plt.plot(xx,rr_interpolated,color='red' )
plt.title('RR interval interpolated')
plt.show()

In [ ]:
# frequency domain feature

In [ ]:
fxx, pxx = signal.welch(x=rr_interpolated, fs=3,nperseg=100)

cond_vlf = (fxx >= 0) & (fxx < 0.04)
cond_lf = (fxx >= 0.04) & (fxx < 0.15)
cond_hf = (fxx >= 0.15) & (fxx < 0.4)
    
    
vlf = trapz(pxx[cond_vlf], fxx[cond_vlf])
lf = trapz(pxx[cond_lf], fxx[cond_lf])
hf = trapz(pxx[cond_hf], fxx[cond_hf])
    
    
total_power = vlf + lf + hf

  
peak_vlf = fxx[cond_vlf][np.argmax(pxx[cond_vlf])]
peak_lf = fxx[cond_lf][np.argmax(pxx[cond_lf])]
peak_hf = fxx[cond_hf][np.argmax(pxx[cond_hf])]

   
lf_nu = 100 * lf / (lf + hf)
hf_nu = 100 * hf / (lf + hf)
lf_nu

In [ ]:
#frequency_domain_features = get_frequency_domain_features(nn_intervals_list)

In [ ]:
#frequency_domain_features

In [ ]:
rr_n = nn_intervals_list[:-1]
rr_n1 =nn_intervals_list[1:]

sd1 = np.sqrt(0.5) * np.std(np.array([rr_n1]) - np.array([rr_n]))
sd2 = np.sqrt(0.5) * np.std(np.array([rr_n1]) + np.array([rr_n]))

s_d=sd1/sd2

In [ ]:
csi_cvi_features=get_csi_cvi_features(nn_intervals_list)
csi_cvi_features

In [ ]:
non_linear_feature=get_poincare_plot_features(nn_intervals_list)
non_linear_feature

In [ ]:
from hrvanalysis import plot_timeseries,plot_distrib,plot_psd,plot_poincare


In [ ]:
plot_timeseries(interpolated_nn_intervals)

In [ ]:
plot_psd(interpolated_nn_intervals,'welch',3,'cubic')